Let's revisit the initial code used to model distributions. This relates to GIt commit 6e0ca5bec5735a5fb19460227df7586a9348ad3c. 

In [282]:
%reset -f
import json
import autopep8

In [283]:
from enum import Enum
from scipy.stats import bernoulli
from dataclasses import dataclass
from typing import Callable


class Distribution(Enum):
    BERNOULLI = 1


distribution_to_ppf = {Distribution.BERNOULLI: bernoulli}

distribution_to_possible_values = {Distribution.BERNOULLI: [0, 1]}


@dataclass
class ConditionalDistribution:
    """Class for representing a conditional distribution."""

    distribution: Distribution
    parameters: dict
    conditionals: [tuple]
    ppf: Callable


It works, but it's a little disjointed, and it's not obvious or easy how we can extend the functionality beyond the present implementation of Bernoulli distributions modelling 0/1 variables. The enumeration and class remain sensible, but the dictionaries represent functionality that can be referenced from the object i.e. the ppf function is already an attribute, while the possible values can be added. 

We update the class definition to add an initialisation function. The function accepts as arguments a distribution enumeration, a dictionary of parameters, and a list of (conditional,value) pairs. The ppf function and possible values are set within the function. The implementation is extended to allow categorical distributions in addition to Bernoulli.

In [284]:
from scipy.stats import rv_discrete
import numpy as np

class Distribution(Enum):
    BERNOULLI = 1
    CATEGORICAL = 2


class ConditionalDistribution:
    """Class for representing a conditional distribution."""

    distribution: Distribution
    parameters: dict
    conditionals: [tuple]
    ppf: Callable
    possible_values: list

    def __init__(self, distribution, parameters, conditionals):
        self.distribution = distribution
        self.parameters = parameters
        self.conditionals = conditionals

        if distribution == Distribution.BERNOULLI:
            # check only one parameter, p
            assert set(parameters.keys()) == {'p'}
            self.ppf = lambda x: bernoulli.ppf(x, **parameters).astype(int)
            self.possible_values = [0, 1]
        elif distribution == Distribution.CATEGORICAL:
            # extract into categories, categories as integers and probabilities
            cats = list(parameters.keys())
            cats_int = range(len(cats))
            probs = list(parameters.values())
            # check probabilities sum to 1 and are individually 0-1
            assert sum(probs) == 1
            assert all([p >= 0 and p < 1 for p in probs])
            # use internal function for integer distribution via rv_discrete
            # when call to ppf is made, use interal distribution's ppf then map back
            # note handling of iterable vs. non-iterable
            cats_dict = dict(zip(cats_int, cats))
            self._dist_int = rv_discrete(values=(cats_int, probs))

            def ppf(x):
                try:
                    iterator = iter(x)
                except TypeError:
                    return cats_dict[self._dist_int.ppf(x)]
                else:
                    return np.array([cats_dict[rv_int] for rv_int in self._dist_int.ppf(x)])
            self.ppf = ppf
            self.possible_values = cats

        else:
            raise AssertionError(
                f"Invalid distribution selected. Valid choices are: {[d.name for d in Distribution]}"
            )

Let's test this using the alarm network and the random number set from the Excel model.

In [285]:
import pandas as pd

with open('../excel/fixed_random_numbers_100k.csv', 'r') as f:
    random_numbers = pd.read_csv(f)
N = random_numbers.shape[0]

random_numbers = 1 - random_numbers  # to align Excel and ppf, since Excel tests lower tail
random_numbers.head()

,B,E,A,M,J
0,0.946102,0.578330,0.741226,0.365127,0.833227
1,0.998269,0.157830,0.723855,0.279586,0.861028
2,0.537858,0.691179,0.728596,0.346864,0.346127
3,0.420817,0.045270,0.658436,0.841218,0.978299
4,0.762577,0.817734,0.442690,0.748229,0.558408


Create condition distributions for the independent node B, one Bernoulli and one categorical using labels 'B' and 'No B'. Generate instances using the ppf functions and check whether the output is consistent i.e. that 1s line up to 'B's and 0s line up to 'No B's.

In [286]:
cd_B_bernoulli = ConditionalDistribution(distribution=Distribution.BERNOULLI,
                                         parameters={'p': 0.001},
                                         conditionals=None)

cd_B_categorical = ConditionalDistribution(distribution=Distribution.CATEGORICAL,
                                           parameters={
                                               'B': 0.001, 'No B': 0.999},
                                           conditionals=None)

rvs_B_bernoulli = cd_B_bernoulli.ppf(random_numbers['B'])
rvs_B_categorical = cd_B_categorical.ppf(1 - random_numbers['B'])
map_cat_bernoulli = {'No B': 0, 'B': 1}
rvs_B_categorical_mapped = np.array(
    [map_cat_bernoulli[rv] for rv in rvs_B_categorical])

assert all(rvs_B_bernoulli == rvs_B_categorical_mapped)

Note that, in calling the ppf function of the categorical distribution, we apply one minus the random numbers. This is due to the convention of the underlying rv_discrete function i.e. 

In [287]:
print(cd_B_bernoulli.ppf(0.0005))
print(cd_B_bernoulli.ppf(0.9995))
print(cd_B_categorical.ppf(0.0005))
print(cd_B_categorical.ppf(0.9995))

0
1
B
No B
